# Labeling all data

In [1]:
import pandas as pd

### Loading and running all classifiers

In [2]:
from __private import fs

In [3]:
print(*fs.list(), sep="\n")

alcohol|accuracy:0.8143360752056404|f1:0.8192219679633866|type:LogisticRegression
alcohol|accuracy:0.8401880141010576|f1:0.8498896247240618|type:SVC
alcohol|accuracy:0.8425381903642774|f1:0.8562231759656651|type:RandomForestClassifier
first_person_label|accuracy:0.5637860082304527|f1:0.5574430033343769|type:SVC
first_person_label|accuracy:0.5637860082304527|f1:0.5643693591852614|type:LogisticRegression
first_person|accuracy:0.6951871657754011|f1:0.8034482758620688|type:RandomForestClassifier
first_person|accuracy:0.7005347593582888|f1:0.7751004016064257|type:LogisticRegression
first_person|accuracy:0.7032085561497327|f1:0.8062827225130889|type:RandomForestClassifier
first_person|accuracy:0.7112299465240641|f1:0.8021978021978021|type:SVC


In [4]:
from classification import dao

In [5]:
%%time
alcohol_classifier = dao.ClassifierAccess.get_byfile(
    "alcohol|accuracy:0.8143360752056404|f1:0.8192219679633866|type:LogisticRegression"
)

firstperson_classifier = dao.ClassifierAccess.get_byfile(
    "first_person|accuracy:0.7112299465240641|f1:0.8021978021978021|type:SVC"
)

firstpersonlevel_classifier = dao.ClassifierAccess.get_byfile(
    "first_person_label|accuracy:0.5637860082304527|f1:0.5643693591852614|type:LogisticRegression"
)

CPU times: user 12.5 s, sys: 29.2 s, total: 41.7 s
Wall time: 1min 14s


In [6]:
from data import DataAccess, LabelGetter

In [ ]:
X = DataAccess.get_as_dataframe()
L = LabelGetter(X)

In [ ]:
%%time
firstpersonlevel_classifier.fit(*L.get_first_person_label())

In [ ]:
%%time
firstperson_classifier.fit(*L.get_first_person())

In [ ]:
del X
del L

# Example

In [ ]:
%%time

from_path = "/Users/JasonLiu/dump/"
to_path = "/Users/JasonLiu/dump/predicted/"

def to_predict(x):
    i, file = x
    df = pd.read_csv(from_path + file).dropna()

    alcohol_predictions = alcohol_classifier.predict_proba(df[["text"]])[:,1]
    df["prediction_alcohol_svc"] = alcohol_predictions

    filter_alcohol = df["prediction_alcohol_svc"] > .75

    df["prediction_firstperson_svc"] = 0
    df.loc[filter_alcohol, "prediction_firstperson_svc"] = firstperson_classifier.predict_proba(
        df[filter_alcohol][["text"]]
    )[:,1]


    firstpersonlevel_predictions = firstpersonlevel_classifier.predict_proba(
        df[filter_alcohol][["text"]]
    )

    df_firstpersonlevel_predictions = pd.DataFrame(
        firstpersonlevel_predictions, columns=[
            "prediction_firstperson_level_0", 
            "prediction_firstperson_level_2", 
            "prediction_firstperson_level_3"],
        index=df[filter_alcohol].index)

    df = df.join(df_firstpersonlevel_predictions).fillna(0)
    df[list(range(1,12))].to_csv(to_path + "predicted_" + file)
    print(i, file)

In [ ]:
from os import listdir
from multiprocessing import Pool

In [ ]:
files = listdir(from_path)

In [ ]:
len(files)

In [ ]:
p = Pool(5)

In [ ]:
%%time
x = p.map(to_predict, enumerate(files))